In [1]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from pyspark.sql import SparkSession

In [2]:
# Paramètres de connexion MySQL
db_url = "jdbc:mysql://localhost:3306/wildlens"
db_properties = { "user": "root",
                  "password": "root",
                  "driver": "com.mysql.cj.jdbc.Driver" }
mysql_driver_path = os.path.abspath("installation/mysql-connector-j-9.1.0.jar")

In [ ]:
# Option 2: Si vous utilisez conda
!conda install -c pytorch torchvision

In [5]:
import os
import sys

# Set Hadoop home environment variable if needed
if sys.platform.startswith('win'):
    os.environ['HADOOP_HOME'] = r'C:\Users\HP\anaconda3\pkgs\pyspark-3.5.3-py312haa95532_0\Lib\site-packages\pyspark\jars\hadoop-client-api-3.3.4.jar'  # Replace with your Hadoop path
else:
    os.environ['HADOOP_HOME'] = '/usr/local/hadoop'  # Replace with your Hadoop path

# Create Spark session with proper configuration
try:
    spark = SparkSession.builder \
        .appName("WildLens ETL - MSPR 24-25") \
        .config("spark.jars", mysql_driver_path) \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.memory", "4g") \
        .getOrCreate()
    
    print("✅ Spark initialisé avec le driver MySQL:", mysql_driver_path)
    print("🔗 Driver chargé:", spark.sparkContext.getConf().get("spark.jars"))
    print("🚀 Version Spark:", spark.version)
    
except Exception as e:
    print("❌ Erreur lors de l'initialisation de Spark:", str(e))
    sys.exit(1)


❌ Erreur lors de l'initialisation de Spark: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: Hadoop home directory C:\Users\HP\anaconda3\pkgs\pyspark-3.5.3-py312haa95532_0\Lib\site-packages\pyspark\jars\hadoop-client-api-3.3.4.jar is not a directory. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1139)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1125)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:489)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$14(Executor.scala:1163)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$14$adapted(Executor.scala:1155)
	at scala.collection.TraversableLike$WithFilter.$anonfun$foreach$

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [17]:
def load_table(table_name):
    return spark.read \
        .format("jdbc") \
        .option("url", db_url) \
        .options(**db_properties) \
        .option("dbtable", table_name) \
        .load()
images_df = load_table("wildlens_images")
facts_df = load_table("wildlens_facts")

Py4JJavaError: An error occurred while calling o156.load.
: java.sql.SQLException: Accès refusé pour l'utilisateur: 'root'@'@localhost' (mot de passe: OUI)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:121)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:114)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:837)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:420)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:238)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:180)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scal

In [ ]:
joined_df = images_df.join(facts_df, images_df.id_espece == facts_df.id_espece) \
                     .select("image", "nom_fr")

df = joined_df.toPandas()

In [ ]:
class WildlensDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None, label2idx=None):
        self.data = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.label2idx = label2idx or {label: idx for idx, label in enumerate(sorted(dataframe['nom_fr'].unique()))}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['image'])
        image = Image.open(img_path).convert('RGB')
        label = self.label2idx[self.data.iloc[idx]['nom_fr']]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['nom_fr'], random_state=42)

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

label2idx = {label: idx for idx, label in enumerate(sorted(df['nom_fr'].unique()))}

train_dataset = WildlensDataset(train_df, 'ressource/image/augmented_train', transform=train_transforms, label2idx=label2idx)
val_dataset = WildlensDataset(val_df, 'ressource/image/augmented_train', transform=val_transforms, label2idx=label2idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(label2idx))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        print(f"[Epoch {epoch+1}] Loss: {total_loss:.4f}, Accuracy: {train_acc:.4f}")

        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
        print(f"Validation Accuracy: {val_correct / val_total:.4f}\n")


In [ ]:
torch.save(model.state_dict(), "wildlens_model.pth")

In [ ]:
print('hello')